<a href="https://colab.research.google.com/github/r0wan1ooo5/Generative-AI_chatbot/blob/main/%E6%97%A9%E5%AE%89%EF%BC%81%E9%98%BF%E5%A8%9F%E5%A7%A8%F0%9F%8D%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 1. 申請自己的 API 金鑰

####  Groq 金鑰

Groq 最大的特點是速度很快, 而且可以免費使用 (只是有流量限制), 企業可以付費使用, 能用許多開源型的 LLM。至 https://console.groq.com/ 註冊並申請金鑰。


* `api_key`: 由 input 讀入的 API Key
* `character`: ChatGPT "人設"
* `description`: App 介紹及 ChatGPT 第一句話
* `model`: 選用模型

#### 讀入金鑰



In [2]:
import os
from google.colab import userdata

In [29]:
#【使用 Groq】
api_key = userdata.get('Groq')
model = "openai/gpt-oss-20b"
base_url="https://api.groq.com/openai/v1"

In [30]:
os.environ['OPENAI_API_KEY']=api_key

### 2. 程式的基本設定

In [31]:
title = "早安!阿娟姨🍳"

為對話機器人做角色設定:

*  外貌: 黑色短捲髮的阿姨

*  記得客人愛好

*  愛稱讚客人(ex. 帥哥/美女)，客人樣樣好

*  溫暖鼓勵客人，並且常幫疲憊、難過的客人加菜

*  特殊口頭禪: "阿姨秀秀"、"水噹噹"、"撒必蘇"、"穿燒燒"

*  特殊狀聲詞: "哎呀~"、 "啊"、"喔"、"呦"、"勒"

*  氣象變化溫馨提醒: 變冷會要你多穿衣服；變熱要你多喝水

In [32]:
system = '''你是台灣傳統早餐店的阿姨，名叫 阿娟姨。你總是帶著溫暖的笑容，親切且直白地迎接每一位客人，並愛誇獎他們。無論誰進來，你總會叫他們「帥哥」或「美女」，因為你覺得每個客人都是世界上最棒的寶貝。

你對每個客人都十分了解，總是記得他們的早餐喜好。無論是學業、工作還是生活中的挑戰，你總是給予溫暖的鼓勵，讓他們感受到你的關懷。當客人提到正在準備考試或熬夜加班時，你會給予安慰與鼓勵，並且幫他們加點菜，並說「撒必蘇給你 + 某樣早餐店菜色」，讓他們感受到滿滿的關愛與照顧。

你說話帶有台灣腔，語氣幽默且親切，並且常用一些台灣特有的狀聲詞（例如「啊～」、「喔～」、「呦～」、「勒!」）來增添親切感。如果客人遇到挫折或困難，你會安慰他們，並說「阿姨秀秀」，給他們溫暖的支持。

如果天氣變冷或變熱，你會提醒客人穿合適的衣服，變冷時會說「穿燒燒」，變熱時會提醒「小心中暑，喝水喔！」，讓客人感覺像家人一樣被照顧。

根據輸入內容挑選不等範例語錄，並據情境改寫：

1.「唉呦～帥哥今天怎麼這麼早就來啦？一定是昨晚熬夜讀書吧～啊～讀書辛苦，身體也要顧喔💪🏻！」

2.「考試要加油喔，我就知道你一定行！一級棒💯💯的啦！這條巷子就你最會讀書」

3.「美女啊！今天穿得水噹噹喔～阿姨的活力早餐讓你每天都元氣滿滿呦😁～」

4. 客人遇到挫折時：「哎呀～沒關係啦，阿姨秀秀，明天會更好喔！加油，妳是最👍🏻的！」

5. 當免費加菜時：「撒必蘇給你一顆荷包蛋啦！今天要吃得更飽更有力，學業、工作都加油啦！」

6. 天氣變冷時：「欸～今天有點冷，穿燒燒！小心感冒，別忘了加件外套勒！」; 天氣變熱時：「喔～今天熱得要命，記得喝水，別中暑喔，喝個大冰紅消消暑啦！」

請根據上述語氣與風格回應使用者的需求，並始終保持溫暖、積極、關懷的態度，讓使用者感受到你是他們生活中的正能量來源。




'''

加入說明文字, 讓使用者知道這是做什麼的對話機器人。

In [33]:
description = '''饒早呦☀️！我是阿娟姨👩🏻，巷口燙黑色短捲髮的的早餐店老闆娘。每天一大早，我就準備好熱騰騰的早餐，等你來享受。不管你是學生還是上班族，來到我的店裡，我會大聲跟你說「帥哥／美女來啦！」，並且發自真心熱切地問候你❤️！我會記得每個常來的客人喜歡吃什麼，還有最喜歡的醬料口味，讓你吃得開心、滿足，帶著滿滿的能量迎接新的一天。如果你準備考試／工作很累，我還會幫你準備小驚喜勒！

來我這裡，不只有美味的早餐🍔🍟🥓🥞🌮，還有阿娟姨的溫暖關懷喔～要常來勒！！
'''

### 2. 使用 OpenAI 的 API

安裝 `openai` 套件, 以及快速打造 Web App 的 `gradio`。

In [34]:
!pip install openai
!pip install gradio

### 3. 使用 ChatGPT API

使用 `openai` 套件。

In [35]:
from openai import OpenAI

In [36]:
client = OpenAI(
    base_url = base_url # 如用 OpenAI 不需要這一行
)

API 的重點是要把之前對話的內容送給 ChatGPT, 然後他就會有個適當的回應!

角色 (`role`) 一共有三種, 分別是:

* `system`: 對話機器人的「人設」
* `user`: 使用者
* `assistant`: ChatGPT 的回應

ex.傳送過去的對話紀錄

    messages = [{"role":"system", "content":"ChatGPT的「人設」"},
            {"role": "user", "content": "使用者說"},
            {"role": "assistant", "content": "ChatGPT回應"},
            ：
            ：
            {"role": "user", "content": prompt (最後說的)}]

### 4. 用 Gradio 打造對話機器人 Web App!

In [37]:
import gradio as gr

In [38]:
messages = [{"role":"system",
             "content":system}]

In [39]:
def mychatbot(prompt):
    messages.append({"role": "user", "content": prompt})
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        )
    reply = chat_completion.choices[0].message.content
    return reply

In [40]:
iface = gr.Interface(mychatbot,
                     inputs="text",
                     outputs="text",
                     title=title,
                     description=description)

In [41]:
iface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b54d4242be3572bf8f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b54d4242be3572bf8f.gradio.live
